In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT",""),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name    = os.getenv('DEPLOYMENT_NAME')

In [2]:
user_msg = """### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### Make a query to list the names of the departments which employed more than 10 employees in the last 3 months.
query:
"""

# user_msg = """### Postgres SQL tables, with their properties:
# #
# # Employee(id, name, department_id)
# # Department(id, name, address)
# # Salary_Payments(id, employee_id, amount, date)
# #
# ### 최근 3 개월간 직원을 10명 이상 뽑은 부서의 이름을 수집하는 쿼리를 만들어줘.
# query: 
# """

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_msg},
    ],
  temperature=0,
  max_tokens=500
)

print(response.choices[0].message.content)

```sql
SELECT d.name AS department_name
FROM Department d
WHERE d.id IN (
    SELECT e.department_id
    FROM Employee e
    WHERE e.id IN (
        SELECT sp.employee_id
        FROM Salary_Payments sp
        WHERE sp.date >= CURRENT_DATE - INTERVAL '3 months'
    )
    GROUP BY e.department_id
    HAVING COUNT(e.id) > 10
);
```


In [3]:
user_msg = """###Can you explain what does this code do? 
# Code: 
SELECT d.name
FROM Department d
WHERE d.id IN (
    SELECT e.department_id
    FROM Employee e
    WHERE e.id IN (
        SELECT sp.employee_id
        FROM Salary_Payments sp
        WHERE sp.date >= current_date - interval '3 months'
    )
    GROUP BY e.department_id
    HAVING COUNT(e.id) > 10
)
# Answer: 
"""

# user_msg = """###이 코드가 무엇을 수행하는지 설명해줘?
# #Code:
# SELECT d.name
# FROM Department d
# WHERE d.id IN (
#     SELECT e.department_id
#     FROM Employee e
#     WHERE e.id IN (
#         SELECT sp.employee_id
#         FROM Salary_Payments sp
#         WHERE sp.date >= current_date - interval '3 months'
#     )
#     GROUP BY e.department_id
#     HAVING COUNT(e.id) > 10
# )
# Answer:
# """
   
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_msg},
    ],
   max_tokens=250)

print(response.choices[0].message.content)

This code is a SQL query that retrieves the names of departments that have more than 10 employees who received a salary payment in the last 3 months.

Here's a breakdown of the code:

1. The outer query selects the `name` column from the `Department` table (aliased as `d`).
2. The `WHERE` clause filters the departments based on their `id`, checking if the department id is present in the subquery result.
3. The subquery in the `IN` condition selects the `department_id` from the `Employee` table (aliased as `e`) where the employee's id is present in another subquery result.
4. The innermost subquery selects the `employee_id` from the `Salary_Payments` table (aliased as `sp`) where the payment date is within the last 3 months.
5. The `GROUP BY` clause groups the employee records by `department_id`.
6. The `HAVING` clause filters the grouped results based on the count of employee ids (more than 10 employees per department).

Overall, this query retrieves the names of departments that have 